In [1]:
!pip install pycryptodome

In [46]:
!pip install rsa

In [5]:
from Crypto.Util import number
import math

In [195]:
import time
import random

start_time = time.time()

def is_prime_miller_rabin(n, k=5):
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True


def get_prime(bits):
    while True:
        candidate = random.getrandbits(bits) | 1
        if is_prime_miller_rabin(candidate):
            return candidate

p = get_prime(1024)
q = get_prime(1024)

while p == q or abs(p - q) == 2:
    q = get_prime(1024)
           
n = p * q

phi_n = (p - 1) * (q - 1)

e = 65537
while math.gcd(e, phi_n) != 1:
    e += 2
    if e == phi_n:
        raise Exception("e is equal phi_n")

d = pow(e,-1,phi_n)

public_key = (e, n)
private_key = (d, n)


file = open("./rsa_small_text.txt", 'rb')
message = file.read()
print(f"Исходное сообщение: {message}")

s_message = [pow(m, e, n) for m in message]
print(f"Зашифрованное сообщение: {bytes().join([s.to_bytes((s.bit_length() + 7) // 8, 'little') for s in s_message])}")

message_from_s = [pow(s, d, n) for s in s_message]
print(f"Выходное сообщение: {bytes().join([s.to_bytes((s.bit_length() + 7) // 8, 'little') for s in message_from_s])}")

print(f"Мой RSA: {time.time() - start_time} seconds")


Исходное сообщение: b'H'
Зашифрованное сообщение: b'G\xcaJ\xc8\x99[\x9a;\x88]\x83\xc2\x90\x80T\x9d\xd9\x1c\x9a\xeew\x17\x88\x9c\xf1\xfd5\xfb[\xc5\xf0\xf1\xc0/j\xba3[@\x87\xee\xb7\x8a\xb6\xbb\x9f\xd6\x0cE\xf5\xe9\x17Xr\x98u\x06\xdc\xe9G\xff\xe9\xe1q\xc3\x92\xfa\xf2\xb3\xf7I\x90\xdf@\x1e\xd8\xaf\xc7j\xb0X\xf8Q\x08k\x8cdD/\x0f.\xe4\xa22h\x8e \xeed)\xbf\x8dm\xcd5\x99w\x8aTO\xdf \x8b\x08(\x84M&\xb1z\xfc}\xd9h\xfcm\xb7\xf6W\x88)\xd2\x98\n\x86\x1b\x04j \xd60\x95\xe33AY\xf0\xc7\xa98\xb2^DbhuF\x96\xc1\xd8\xf4\xff\x16*\x0f\xeb\xed\xd9j\x1d\xdb^(f\xf6g\x0c\x87gG\x88\xf0I\x1b\x18\x0f\x13p\xec\xb0\xe3\xae\xeb\x1c\x8a\x91\x167\xc0/\xae7\xce-\xfc9\n!MD]\xaaj\x16\x92\x96\xbeg`\\\xf5\xa3s\xc9S]l\x13\x18\x8fn\x1e\xf8\xd7V\n<\x8ba\xe7\xc2\xd3K\xd0\xe7U\xbb]\xb5\x1d\x16e\x87\x9f\xe2'
Выходное сообщение: b'H'
Мой RSA: 3.9699416160583496 seconds


In [197]:
import rsa

start_time = time.time()

public_key, private_key = rsa.newkeys(1024)
file = open("./rsa_small_text.txt", 'rb')
message = file.read()
print(f"Исходное сообщение: {message}")

ciphertext = rsa.encrypt(message, public_key)
print(f"Зашифрованное сообщение: {ciphertext}")

decrypted_message = rsa.decrypt(ciphertext, private_key)
print(f"Выходное сообщение: {decrypted_message}")

print(f"RSA: {time.time() - start_time} seconds")

Исходное сообщение: b'H'
Зашифрованное сообщение: b'\xa6\t\x1ct\xc4\xf1\xafi#\xdb\x84\xe0\xa1\x92\xda\xb4\xf9\xc8u\xfd\xf1\xdd\xa6\xed\x9b`\xad\x1fo\xd6}y\xaf\xcf0\xbf\x10\xc3\xe68\xc8@n\xc9\x1a\x1a\xfe\x1e\xb7\x9d\x96>\x08P\xd6G\x13\xf7\xf7Yv`\xb2B\x12\x06~\xc3\x1aTb\xbd\x90\xc4\xcc\xb1\xabh:\xa3\xddD\xf3\x08V\xda\\\x92\x1f\x88\xc8\xde\x06 \xa9\x1f3\x89\x14g\x7f\x85\xbf\xf6\xa4\x0eL\xa9\xcd\xbc\x06\xef]]\x15E\x85=g22\x8a5\x168K\xc0a'
Выходное сообщение: b'H'
RSA: 0.9050173759460449 seconds


In [190]:
import hashlib


def encrypt(hash_message, d, n):
    return [pow(m, d, n) for m in hash_message]

def sign(message, private_key):
    sha256 = hashlib.sha256()
    sha256.update(message)
    hash_message = sha256.digest()

    d = private_key[0]
    n = private_key[1]
    s_message = encrypt(hash_message, d, n)
    return hash_message, s_message

def decrypt(s_message, e, n):
    message_from_s = [pow(s, e, n) for s in s_message]
    return bytes().join([s.to_bytes((s.bit_length() + 7) // 8, 'little') for s in message_from_s])

def verify(hash_message, s_message, public_key):
    e = public_key[0]
    n = public_key[1]
    message_from_s = decrypt(s_message, e, n)
    if message_from_s == hash_message:
        return True
    else:
        return False

public_key = (e, n)
private_key = (d, n)

file = open("./big.txt", 'rb')
message = file.read()

hash_message, s_message = sign(message, private_key)

auth = verify(hash_message, s_message, public_key)

if auth == True:
    print("Подпись подтверждена")

Подпись подтверждена
